In [120]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [121]:
athletes = pd.read_csv('athlete_events.csv')
athletes.shape

(271116, 15)

In [122]:
Population = pd.read_csv('WorldPopulation.csv')
Population.shape

(217, 62)

In [123]:
Population.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 61'], axis = 1, inplace=True)

In [124]:
Population.sample(3)

,Country,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
121,Monaco,MCO,22452.0,22808.0,23039.0,23168.0,23236.0,23282.0,23305.0,23292.0,...,35111.0,35853.0,36534.0,37094.0,37497.0,37783.0,37971.0,38132.0,38307.0,38499
101,Kyrgyz Republic,KGZ,2172300.0,2255900.0,2333400.0,2413700.0,2495300.0,2573300.0,2655300.0,2736500.0,...,5268400.0,5318700.0,5383300.0,5447900.0,5514600.0,5607200.0,5719600.0,5835500.0,5956900.0,6082700
151,Panama,PAN,1132921.0,1167035.0,1202373.0,1238823.0,1276276.0,1314626.0,1353804.0,1393799.0,...,3453807.0,3516268.0,3579385.0,3643222.0,3707782.0,3772938.0,3838462.0,3903986.0,3969249.0,4034119


In [125]:
athletes.sample(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
191783,96289,Adrin Portela,M,30.0,185.0,95.0,Argentina,ARG,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Men's Handball,NaN
229149,115053,Elfriede Steurer (-Reichert),F,27.0,NaN,NaN,Austria,AUT,1952 Summer,1952,Summer,Helsinki,Athletics,Athletics Women's 80 metres Hurdles,NaN
68540,34991,Jos Pinto Ferreira,M,29.0,181.0,79.0,Portugal,POR,1952 Summer,1952,Summer,Helsinki,Fencing,"Fencing Men's Sabre, Team",NaN


In [126]:
city = athletes['City'] == 'Beijing'
athletes[city].head(3)
athletes[city].shape


(13602, 15)

In [127]:
Population = pd.melt(Population, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population' )
Population.head()

,Country,Country Code,Year,Population
0,Aruba,ABW,1960,54211.0
1,Afghanistan,AFG,1960,8996351.0
2,Angola,AGO,1960,5643182.0
3,Albania,ALB,1960,1608800.0
4,Andorra,AND,1960,13411.0


In [128]:
Population['Year'] = pd.to_numeric(Population['Year'])


In [129]:
Population.head(3)

,Country,Country Code,Year,Population
0,Aruba,ABW,1960,54211.0
1,Afghanistan,AFG,1960,8996351.0
2,Angola,AGO,1960,5643182.0


In [130]:
#a = athletes.set_index('NOC').join(population_2008.set_index('Country Code'), how='inner')

In [131]:
Population = pd.merge(Population, athletes, left_on = ['Country Code', 'Year'], right_on = ['NOC', 'Year'])

In [132]:
Population.shape

(151931, 18)

In [133]:
Population.head(4)

,Country,Country Code,Year,Population,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Event,Medal
0,Afghanistan,AFG,1960,8996351.0,29626,Sultan Mohammad Dost,M,28.0,168.0,73.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Welterweight, Freestyle",NaN
1,Afghanistan,AFG,1960,8996351.0,39922,Abdul Ghafar Ghafoori,M,22.0,172.0,70.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Athletics,Athletics Men's 4 x 100 metres Relay,NaN
2,Afghanistan,AFG,1960,8996351.0,43901,Ghulam Mohiddin Gunga,M,26.0,168.0,81.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Light-Heavyweight, Freestyle",NaN
3,Afghanistan,AFG,1960,8996351.0,58364,Mohammad Ibrahim Kederi,M,20.0,166.0,62.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Featherweight, Freestyle",NaN


In [134]:
Population = Population.drop( ['ID','NOC','Games'], axis=1)

In [135]:
print(Population.isnull().sum())

Country              0
Country Code         0
Year                 0
Population           0
Name                 0
Sex                  0
Age                545
Height            7426
Weight            8098
Team                 0
Season               0
City                 0
Sport                0
Event                0
Medal           132298
dtype: int64


In [136]:
Population = Population[Population['Age'].notna()]
Population = Population[Population['Weight'].notna()]
Population = Population[Population['Height'].notna()]
Population['Medal'].fillna('DNW', inplace = True)

In [137]:
print(Population.isnull().sum())

Country         0
Country Code    0
Year            0
Population      0
Name            0
Sex             0
Age             0
Height          0
Weight          0
Team            0
Season          0
City            0
Sport           0
Event           0
Medal           0
dtype: int64


In [138]:
Population.shape

(143142, 15)

In [139]:
#if athlete won medals then exceed 1, if not tehn exceeds 0
Population['Medal_Won'] = np.where(Population.loc[:,'Medal'] == 'DNW', 0, 1)

    

In [140]:
Population.to_csv('athletePopulation.csv', sep=',')